<a href="https://colab.research.google.com/github/ucfilho/ANN/blob/master/volume_laranja/frutas_laranja_parte_01_nov_25_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# parte 1- le os arquivos e separa eles em a e b (a frutas deitadas e b sao as frusta em pe)
# parte 2- usa o filtro
# parte 3- calcula o volume : para um objeto
# parte 4- para todos objetos
# parte 5- explora tecnicas para melhorar o volume
# parte 6 - adiciona o volume falso como coluna para X tentativa de melhorar calc
# parte 7 - inicia calcular as areas
# parte 8- identica a parte 7 mas em rotina para duas orientacoes (up e down)
# parte 9 - ANN com as novas variaveis de area

In [2]:
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np
import pandas as pd
import os 
import cv2
import re, random
from skimage import feature
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import random
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score 

In [3]:
!gdown --id 13zIVJBDpB-0cNhdR3WFNNQCR91DHoGO-

Downloading...
From: https://drive.google.com/uc?id=13zIVJBDpB-0cNhdR3WFNNQCR91DHoGO-
To: /content/laranjas.zip
100% 1.10G/1.10G [00:06<00:00, 168MB/s]


In [4]:
#https://drive.google.com/drive/folders/1CIzevQ1oFhUapUNMuVXaYJbbqxinCRAu?usp=sharing
# https://drive.google.com/file/d/13zIVJBDpB-0cNhdR3WFNNQCR91DHoGO-/view?usp=sharing

In [5]:
import zipfile
file_name = zipfile.ZipFile('laranjas.zip', 'r')
file_name.extractall()

In [6]:
!ls

100a.png  21a.png  33a.png  45a.png  57a.png  69a.png  80a.png	92a.png
100b.png  21b.png  33b.png  45b.png  57b.png  69b.png  80b.png	92b.png
10a.png   22a.png  34a.png  46a.png  58a.png  6a.png   81a.png	93a.png
10b.png   22b.png  34b.png  46b.png  58b.png  6b.png   81b.png	93b.png
11a.png   23a.png  35a.png  47a.png  59a.png  70a.png  82a.png	94a.png
11b.png   23b.png  35b.png  47b.png  59b.png  70b.png  82b.png	94b.png
12a.png   24a.png  36a.png  48a.png  5a.png   71a.png  83a.png	95a.png
12b.png   24b.png  36b.png  48b.png  5b.png   71b.png  83b.png	95b.png
13a.png   25a.png  37a.png  49a.png  60a.png  72a.png  84a.png	96a.png
13b.png   25b.png  37b.png  49b.png  60b.png  72b.png  84b.png	96b.png
14a.png   26a.png  38a.png  4a.png   61a.png  73a.png  85a.png	97a.png
14b.png   26b.png  38b.png  4b.png   61b.png  73b.png  85b.png	97b.png
15a.png   27a.png  39a.png  50a.png  62a.png  74a.png  86a.png	98a.png
15b.png   27b.png  39b.png  50b.png  62b.png  74b.png  86b.png	98b.png
16a.pn

In [7]:
FILE='laranjas.zip'
img_name=[]
x_original = [] 

ref ='png'
data_file ='xlsx'

with zipfile.ZipFile(FILE, "r") as f:
    for name in f.namelist():
      if(name[-3:]==ref):
        img_name.append(name)
        #x_original.append(cv2.resize(cv2.imread(name),(60,80)))
        x_original.append(cv2.resize(cv2.imread(name,0),(60,80)))
      else:
        if(name[-4:] == data_file):
          df =pd.read_excel(name)


In [8]:
name_down=[]  # down representa as frutas deitadas
original_down = [] 
name_up=[] # down representa as frutas em pe
original_up= [] 

In [9]:
k = 0
for name in img_name:
  if(name[-5]=='a'):
    name_up.append(name)
    original_up.append(x_original[k])
  else:
    name_down.append(name)
    original_down.append(x_original[k])
  k = k +1


In [10]:
def get_dimension(img):
  im = img
  # Compute the Canny filter for two values of sigma
  edges1 = feature.canny(im)
  edges2 = feature.canny(im, sigma=3)
  nrows,ncols =edges2.shape
  Foto = np.ones((nrows,ncols))*255
  for i in range(nrows):
    for j in range(ncols):
      if edges2[i,j] == True:
        Foto[i,j]=0
  Foto = Foto.astype(int)
# calculo do raio horizontal
  Quadrado = []
  Raio = []
  for i in range(nrows):
    values = np.zeros(4)
    k = -1
    for j in range(1,ncols):
      if(Foto[i,j-1]==255 and Foto[i,j]==0):
        if(k<3):
          k = k+1
          values[k]=j
    L = values[3] - values[0]
    d = values[2] - values[1]
    Quadrado.append(L)
    Raio.append(d)
  L = np.array(Quadrado).max()
  Raio = np.array(Raio).max()
  medida = Raio/L*14.5
  return medida,Foto


In [11]:
def area_fruta(img):
  (rows, cols) =img.shape

  img_fruit = np.zeros(img.shape,np.uint8) 

  for i in range(rows):
    for j in range(cols):
      if(img[i,j]>100):
        img[i,j]=0


  for i in range(rows):
    for j in range(cols):
      if(img[i,j]>0):
        img[i,j]=255
        img_fruit[i,j]=255

  xi=0;xf=0;yi=0;yf=0;
  jref=int(cols/2)
  iref=int(rows/2)

  for i in range(rows):
    j = jref
    if(img[i,j]>0):
      if(xi==0):
        xi = i
      else:
        xf = i
  for j in range(cols):
    i = iref
    if(img[i,j]>0):
      if(yi==0):
        yi = j
      else:
        yf = j

  for i in range(rows):
    for j in range(cols):
      if(i > xi and i < xf and j>yi and j < yf):
        img[i,j]=255

    area = 1-np.sum(img_fruit.ravel())/np.sum(img.ravel())

  return area

In [12]:
(num,l,w)=np.array(original_down).shape

In [13]:
dimensao =[]
for i in range(num):
  lista =[]
  Ind = i
  img= original_up[Ind]
  medida1,foto = get_dimension(img)
  lista.append(medida1)
  img = img.transpose()
  medida2,foto = get_dimension(img)
  lista.append(medida2)
  img2= original_down[Ind]
  medida3,foto = get_dimension(img2)
  lista.append(medida3)
  img2 = img2.transpose()
  medida4,foto = get_dimension(img2)
  lista.append(medida4)
  dimensao.append(lista)


In [14]:
area_up =[]
area_down =[]
for i in range(num):
  lista =[]
  Ind = i
  img= original_up[Ind]
  area = area_fruta(img)
  area_up.append(area)
  img= original_down[Ind]
  area = area_fruta(img)
  area_down.append(area)

In [15]:
print(area_up)

[0.18342749529190205, 0.18333333333333335, 0.1645427286356822, 0.19939462731744229, 0.14736042537030003, 0.1845979614949037, 0.16314996232102486, 0.18460955269143287, 0.18962906888720665, 0.18597560975609762, 0.18704484476811034, 0.17095310136157338, 0.1796610169491526, 0.15149166989538942, 0.17938066465256797, 0.17309896828429494, 0.20190839694656493, 0.16085489313835766, 0.1797795515013303, 0.19380430676237248, 0.19506726457399104, 0.21926536731634183, 0.19528875379939215, 0.19153758972421608, 0.21595184349134688, 0.17524602573807724, 0.18209067066316975, 0.2058047493403694, 0.1532288880397401, 0.17820658342792284, 0.1573506891271057, 0.17890772128060262, 0.21020793950850658, 0.18577374195989405, 0.17850432168357755, 0.16196918451709885, 0.16405082212257105, 0.1927574500188608, 0.15542857142857147, 0.15462373642830396, 0.19045771916214116, 0.18783679753656657, 0.1822306238185255, 0.1664787673806839, 0.1656580937972768, 0.14704747201852564, 0.1823015571591341, 0.18311981914091935, 0.1

In [16]:
print(area_down)

[0.19419306184012064, 0.20463173880030372, 0.17516879219804948, 0.21928817451205507, 0.15305734903152302, 0.1969924812030075, 0.1879271070615034, 0.21044663133989405, 0.20204313280363229, 0.20839636913767023, 0.21054672353625437, 0.18367346938775508, 0.21030372933487118, 0.20847457627118648, 0.19871794871794868, 0.19358490566037734, 0.215228043724086, 0.17334331710969675, 0.1894054878048781, 0.20620506999621646, 0.2092675635276532, 0.23358490566037737, 0.218595825426945, 0.2108046845485455, 0.2269823374671176, 0.18631897203325776, 0.20090463626083677, 0.23024574669187148, 0.16488140780413163, 0.18205904617713853, 0.1732616212063004, 0.1902604756511891, 0.22332317073170727, 0.19885057471264367, 0.1840796019900498, 0.18707738542449281, 0.18164794007490637, 0.20581570996978849, 0.16660355925785686, 0.17199849454271732, 0.20219198790627357, 0.1973634651600753, 0.18583739228175344, 0.18071833648393199, 0.18216463414634143, 0.15674453654860587, 0.1923509561304837, 0.200376647834275, 0.171222

In [17]:
def elipsoide(valores):
  dif = np.zeros(4)
  k = 0
  ref =1E99
  for i in range(2):
    for j in range(2):
      dif[k] = abs(valores[i]-valores[j+2])
      if(ref > dif[k]):
        ref = dif[k]
        ind1=i # 0,1
        ind2=j+2 # 2,3
      k = k+1
  b = (valores[ind1]+valores[ind2])/2
  if(ind1 ==0):
    a = valores[1]
  else:
    a = valores[0]
  if(ind2 ==2):
    c = valores[3]
  else:
    c = valores[2]
  Vol = 4/3*np.pi*a*b*c

  return Vol  
  

  

In [18]:
Volume = []
X =[]
for i in range(num):
  valores = dimensao[i]
  X.append(valores)
  Vol = elipsoide(valores)
  Volume.append(Vol)

In [19]:
#dimensao[0]

In [20]:
y = df['Volume'].values
#print(y)

NameError: ignored

In [ ]:
X = np.array(X)
#X =np.concatenate((X,np.array(Volume).reshape(num,1)),axis=1)
X =np.concatenate((X,np.array(area_up).reshape(num,1)),axis=1)
X =np.concatenate((X,np.array(area_down).reshape(num,1)),axis=1)
y = np.array(y)
y = y.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.10, shuffle=True, random_state=3)# 1

In [ ]:
scaler_X = preprocessing.MinMaxScaler((-1,1)).fit(X)
scaler_Y = preprocessing.MinMaxScaler((-1,1)).fit(y)

In [ ]:
X_train = scaler_X.transform(X_train)
X_test = scaler_X.transform(X_test)
y_train = scaler_Y.transform(y_train)
y_test = scaler_Y.transform(y_test)

In [ ]:
# define a rede neural 
N_Hidden= 20
model = MLPRegressor(solver='lbfgs',activation='tanh',alpha=1e-5,hidden_layer_sizes=(N_Hidden), max_iter=1500,random_state=11 )

In [ ]:
# treina a rede neural
model.fit(X_train, y_train)

In [ ]:
# usa a rede neural treina em calculos para treino e teste
y_calc_train=model.predict(X_train)
y_calc_test=model.predict(X_test)

In [ ]:
# transforma dos dados calculados na forma original (nao normatizado)
y_obs_train=scaler_Y.inverse_transform(np.array(y_train).reshape(-1,1))
y_obs_test=scaler_Y.inverse_transform(np.array(y_test).reshape(-1,1))
y_calc_train=scaler_Y.inverse_transform(y_calc_train.reshape(-1,1))
y_calc_test=scaler_Y.inverse_transform(y_calc_test.reshape(-1,1))

In [ ]:
print(pd.DataFrame({'test calc':y_calc_test.ravel(),'test true':y_obs_test.ravel()}))

In [ ]:
# expressa os dois dados em figura
fig = plt.figure()
ax1 = fig.add_subplot(111)
ax1.scatter(y_obs_test,y_calc_test, s=10, c='b', marker="s", label='teste')
ax1.scatter(y_obs_train,y_calc_train, s=10, c='r', marker="o", label='treino')
plt.legend(loc='upper left')
#plt.xlim([15,30])
#plt.ylim([15,30])
plt.ylabel('Y Calc')
plt.xlabel('Y  Obs')
plt.show()

In [ ]:
mse=mean_squared_error(y_obs_test,y_calc_test)
print("MSE teste=",mse)
R2=r2_score(y_obs_test,y_calc_test)
print("R^2 teste=",R2)

In [ ]:
mse=mean_squared_error(y_obs_train,y_calc_train)
print("MSE treino=",mse)
R2=r2_score(y_obs_train,y_calc_train)
print("R^2 treino=",R2)

In [ ]:
area_real = []
for i in area_down:
  area_real.append(i* 14.5* 14.5) 
plt.plot(area_real)
plt.xlabel('ensaio')
plt.ylabel('Area(cm2)')
print('area media down',np.array(area_real).mean())
print('area desvio down',np.array(area_real).std())

In [ ]:
area_real = []
for i in area_up:
  area_real.append(i* 14.5* 14.5) 
plt.plot(area_real)
plt.xlabel('ensaio')
plt.ylabel('Area(cm2)')
print('area media up',np.array(area_real).mean())
print('area desvio up',np.array(area_real).std())

In [ ]:
np.array(area_real).mean()

In [ ]:
plt.plot(area_up)